# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [3]:
%%wdsparql

SELECT ?date ?lugarLabel ?paisLabel
WHERE
{
    wd:Q34981 wdt:P569 ?date.
    wd:Q34981 wdt:P19 ?lugar.
    ?lugar wdt:P17 ?pais
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,date,lugarLabel,paisLabel
0,1920-01-02T00:00:00Z,Petrovichi,Russia


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [40]:
%%wdsparql
SELECT ?ocupaciones ?ocupacionesLabel
WHERE {
    wd:Q34981 wdt:P106 ?ocupaciones.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?ocupacionesLabel

,ocupaciones,ocupacionesLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [6]:
%%wdsparql

SELECT ?escritorLabel
WHERE
{
    wd:Q34981 wdt:P106 ?escritor. ## P106 = ocupación 
    ?escritor wdt:P279+ wd:Q36180. ## P279 = subclase de
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,escritorLabel
0,journalist
1,science writer
2,novelist
3,prosaist
4,non-fiction writer
5,screenwriter
6,science fiction writer
7,autobiographer
8,scientist
9,biochemist


### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [42]:
%%wdsparql

SELECT ?esposa ?esposaLabel ?start ?fin
WHERE{
    # Isaac Asimov casado con []
    # [esta esposa; con esta fecha de inicio; con esta fecha final]
    wd:Q34981 p:P26 [ps:P26 ?esposa; pq:P580 ?starq; pq:P582 ?finq].
    
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
    # Para mejorar la legibilidad de la fecha
    BIND( concat( str(year(?starq)), "-", str(month(?starq)), "-", str(day(?starq)) ) as ?start )
    BIND( concat( str(year(?finq)), "-", str(month(?finq)), "-", str(day(?finq)) ) as ?fin )
}
ORDER BY ASC (?start)

,esposa,esposaLabel,start,fin
0,http://www.wikidata.org/entity/Q107693884,Gertrude Asimov,1942-7-26,1973-1-1
1,http://www.wikidata.org/entity/Q517435,Janet Asimov,1973-1-1,1992-1-1


### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [14]:
%%wdsparql

SELECT ?premioLabel ?fechaLabel ?obraLabel
WHERE
{
    wd:Q34981 p:P166 [ ps:P166 ?premio ].
    OPTIONAL { wd:Q34981 p:P166 [ ps:P166 ?premio; pq:P1686 ?obra ]. }
    OPTIONAL { wd:Q34981 p:P166 [ ps:P166 ?premio; pq:P585 ?fecha ]. }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
    # Para mejorar la legibilidad de la fecha, aunque solo está disponible el año
    BIND( concat( str(year(?fecha))) as ?fechaLabel )
}
ORDER BY ?fecha

,premioLabel,fechaLabel,obraLabel
0,AAAS Fellow,NaN,None
1,Fellow of the Committee for Skeptical Inquiry,NaN,None
2,Hugo Award,1963.0,Foundation Trilogy
3,Hugo Award,1963.0,Foundation Trilogy
4,James T. Grady-James H. Stack Award for Interp...,1965.0,None
5,Hugo Award,1966.0,Foundation Trilogy
6,Hugo Award,1966.0,Foundation Trilogy
7,Edward E. Smith Memorial Award,1967.0,None
8,Nebula Award for Best Novel,1972.0,The Gods Themselves
9,Hugo Award for Best Novel,1973.0,The Gods Themselves


### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [15]:
%%wdsparql

SELECT ?obra ?obraLabel 
WHERE{
    ?obra wdt:P136/wdt:P279* wd:Q3238422. # instancia de subclases de obras literarias de ciencia ficcion
    ?obra wdt:P50 wd:Q34981.
     
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        
}ORDER BY ?obraLabel

,obra,obraLabel
0,http://www.wikidata.org/entity/Q603705,. . . That Thou Art Mindful of Him
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.
2,http://www.wikidata.org/entity/Q4657902,A Loint of Paw
3,http://www.wikidata.org/entity/Q3549799,A Statue for Father
4,http://www.wikidata.org/entity/Q3618113,Anniversary
5,http://www.wikidata.org/entity/Q2872379,Author! Author!
6,http://www.wikidata.org/entity/Q4920795,Black Friar of the Flame
7,http://www.wikidata.org/entity/Q4925104,Blank!
8,http://www.wikidata.org/entity/Q3511886,Blind Alley
9,http://www.wikidata.org/entity/Q2855170,Breeds There a Man...?


### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [28]:
%%wdsparql

SELECT ?obraLabel (MIN(?fechaQ) AS ?primeraFecha)
WHERE
{
    ?obra wdt:P136/wdt:P279* wd:Q3238422; # P136 = género ; P279 = Subclase de ; Q3238422 = obras literarias de ciencia ficcion
          wdt:P50 wd:Q34981. # P50 = autor
    OPTIONAL { ?obra wdt:P577 ?fecha } # P577 = fecha de publicación
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
    BIND( concat( str(year(?fecha)), "-", str(month(?fecha))) as ?fechaQ ) # Tan solo mostramos el mes y el año
    
}
GROUP BY ?obraLabel
ORDER BY ?fecha

,obraLabel,primeraFecha
0,Foundation series,None
1,Satisfaction Guaranteed,None
2,Foundation Trilogy,None
3,Sucker Bait,1954-1
4,The Ugly Little Boy,1958-1
5,Nemesis,1989-1
6,Robbie,1940-9
7,"I, Robot",1950-12
8,Light Verse,1973-1
9,Risk,1955-5


### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [29]:
%%wdsparql

SELECT (COUNT(*) AS ?Numero_de_obras)
WHERE
{
    SELECT DISTINCT ?obra
    WHERE{
        ?obra wdt:P31 wd:Q7725634; # instancia de literary work
              wdt:P50 wd:Q34981 # tiene como autor a Isaac Asimov
    }
}

,Numero_de_obras
0,276


### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [30]:
%%wdsparql

SELECT ?obraLabel ?publicacion
WHERE
{
    
  {
    SELECT DISTINCT ?obra (min(?fechaQ) AS ?publicacion)
    WHERE
    {
      ?obra wdt:P31 wd:Q7725634; # P31 = instancia de ; Q7725634 = trabajo literario
             wdt:P50 wd:Q34981; # P50 = autor
             wdt:P577 ?fecha.
      FILTER( (year(?fecha) >= 1970) && (year(?fecha) <= 1980)). # P577 = fecha de publicación
        
      BIND( concat( str(year(?fecha)), "-", str(month(?fecha)), "-", str(day(?fecha)) ) as ?fechaQ )
    }
    GROUP BY ?obra
      
  
  }
            
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
}
ORDER BY ?publicacion

,obraLabel,publicacion
0,Asimov's Guide to Shakespeare,1970-1-1
1,2430 A. D.,1970-1-1
2,Waterclap,1970-1-1
3,The Solar System and Back,1970-1-19
4,As Chemist to Chemist,1970-5-1
5,The Stars in their Courses,1971-1-1
6,Q16652396,1972-1-1
7,The Left Hand of the Electron,1972-1-1
8,The Greatest Asset,1972-1-1
9,Take a Match,1972-1-1


### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

In [98]:
%%wdsparql

SELECT ?escritor ?escritorLabel ?nacimiento ?defuncion
WHERE
{
    
    SELECT DISTINCT ?escritor ?escritorLabel ?nacimiento ?defuncion ?centro
    WHERE {
        
        FILTER((?escritor) != wd:Q34981). # no tenemos en cuenta a asimov
        ?escritor wdt:P106 wd:Q18844224;
                  wdt:P69 ?centro.
        wd:Q34981 wdt:P69 ?centro.
        
        
        OPTIONAL{?escritor wdt:P569 ?nacimiento.}
        OPTIONAL{?escritor wdt:P570 ?defuncion.}
        
    
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    
    }
}
ORDER BY (?centro) (?escritorLabel)

,escritor,escritorLabel,nacimiento,defuncion
0,http://www.wikidata.org/entity/Q13563168,Alaya Dawn Johnson,1982-03-31T00:00:00Z,None
1,http://www.wikidata.org/entity/Q547414,Algis Budrys,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z
2,http://www.wikidata.org/entity/Q4800570,Arthur W. Saha,1923-10-31T00:00:00Z,1999-11-19T00:00:00Z
3,http://www.wikidata.org/entity/Q4886274,Ben Parris,1961-01-01T00:00:00Z,None
4,http://www.wikidata.org/entity/Q5233934,David G. Hartwell,1941-07-10T00:00:00Z,2016-01-20T00:00:00Z
5,http://www.wikidata.org/entity/Q551929,David Kyle,1919-02-14T00:00:00Z,2016-09-18T00:00:00Z
6,http://www.wikidata.org/entity/Q5294012,Donald Barr,1921-08-08T00:00:00Z,2004-02-05T00:00:00Z
7,http://www.wikidata.org/entity/Q5297967,Doris Pitkin Buck,1898-01-03T00:00:00Z,1980-12-04T00:00:00Z
8,http://www.wikidata.org/entity/Q1778440,Dorothy B. Hughes,1904-08-10T00:00:00Z,1993-05-06T00:00:00Z
9,http://www.wikidata.org/entity/Q548140,Eric Temple Bell,1883-02-07T00:00:00Z,1960-12-21T00:00:00Z


### Consulta 11

¿Cuántos asertos hay sobre Asimov en Wikidata? Ten en cuenta que Asimov puede aparece tanto como sujeto como objeto de cada tripleta.

In [49]:
%%wdsparql

SELECT (COUNT(*) AS ?numAsertos)
WHERE {
    {
        SELECT ?x ?y 
        WHERE{
            ?x ?y wd:Q34981
        }
    } UNION{
        SELECT ?x ?y
        WHERE{
            wd:Q34981 ?x ?y
        } 
    }
}

,numAsertos
0,2277


__Fecha de las consultas: 21 de abril de 2022__